# AgentForce SDK - Create Agent Examples

This notebook demonstrates different ways to create agents using the AgentForce SDK.

## Prerequisites

First, let's install the AgentForce SDK:

In [ ]:
# Install the latest version of AgentForce SDK
!pip install ../dist/agentforce_sdk-0.1.4-py3-none-any.whl

## Setup

Let's set up our credentials for AgentForce:

In [ ]:
import os
from agent_sdk import Agentforce, AgentUtils
from agent_sdk.models.agent import Agent
from agent_sdk.models.topic import Topic
from agent_sdk.models.action import Action
from agent_sdk.models.input import Input
from agent_sdk.models.output import Output
from agent_sdk.models.system_message import SystemMessage
from agent_sdk.models.variable import Variable
from agent_sdk.core.auth import BasicAuth

# Replace with your Salesforce credentials
username = "your_username"
password = "your_password"

auth = BasicAuth(username=username, password=password)

# Initialize the AgentForce client
agentforce = Agentforce(auth=auth)

## Method 1: Create Agent from JSON File

This is the simplest way to create an agent, by using a pre-defined JSON file.

In [ ]:
# Path to JSON file
json_file_path = "../assets/input.json"

# Create the agent from JSON file
agent = AgentUtils.create_agent_from_file(json_file_path)

# View the agent configuration
print(f"Agent Name: {agent.name}")
print(f"Description: {agent.description}")
print(f"Company: {agent.company_name}")
print(f"Topics: {len(agent.topics)}")

# To deploy the agent:
# result = agentforce.create(agent)
# print(f"Agent created successfully")

## Method 2: Create Agent from Directory Structure

This method uses a modular directory structure where each component is defined in a separate file.

In [ ]:
# Path to directory structure
directory_path = "../assets/chained_agent_dir"
agent_name = "order_management_agent"

# Create the agent from directory structure
agent = AgentUtils.create_agent_from_directory_structure(directory_path, agent_name)

# View the agent configuration
print(f"Agent Name: {agent.name}")
print(f"Description: {agent.description}")
print(f"Topics: {len(agent.topics)}")

# Display topic information
for i, topic in enumerate(agent.topics):
    print(f"\nTopic {i+1}: {topic.name}")
    print(f"Description: {topic.description}")
    print(f"Actions: {len(topic.actions) if topic.actions else 0}")
    
    if topic.actions:
        for j, action in enumerate(topic.actions):
            print(f"  Action {j+1}: {action.name}")

# To deploy the agent:
# result = agentforce.create(agent)
# print(f"Agent created successfully")

## Method 3: Create Agent Programmatically

This method demonstrates how to create an agent by constructing it programmatically with objects.

In [ ]:
# Create an action with inputs and outputs
action = Action(
    name="findOrder",
    description="Find order details using an order ID",
    inputs=[
        Input(
            name="orderID",
            description="Order identification number",
            data_type="string"
        )
    ],
    outputs=[
        Output(
            name="orderDetails",
            description="Details of the order",
            data_type="object"
        )
    ]
)

# Create a topic with the action
topic = Topic(
    name="Order Management",
    description="Handles all user requests related to finding and managing orders",
    scope="public",
    instructions=[
        "If a user cannot find their order, attempt to locate it using the order ID",
        "If a user wants to check the status of their order, retrieve the order details"
    ],
    actions=[action]
)

# Create the agent with the topic
agent = Agent(
    name="ProgrammaticAgent",
    description="An agent created programmatically for order management",
    agent_type="External",
    agent_template_type="EinsteinServiceAgent",
    company_name="Example Corp",
    sample_utterances=[
        "What's the status of my order?",
        "I need to find my order"
    ],
    system_messages=[
        SystemMessage(message="Welcome to Order Management!", msg_type="welcome"),
        SystemMessage(message="I'm sorry, I encountered an error.", msg_type="error")
    ],
    variables=[
        Variable(name="apiKey", data_type="string", default_value="sample-key")
    ],
    topics=[topic]
)

# View the agent configuration
print(f"Agent Name: {agent.name}")
print(f"Description: {agent.description}")
print(f"Topics: {len(agent.topics)}")

# To deploy the agent:
# result = agentforce.create(agent)
# print(f"Agent created successfully")

## Method 4: Create & Export Agent Directory Structure

This method demonstrates how to create an agent programmatically and then export it to a directory structure.

In [ ]:
# We'll use the same agent we created programmatically above
export_dir = "./exported_agent"

# Create the directory structure
AgentUtils.create_agent_directory_structure(export_dir, agent)

print(f"Agent exported to directory: {export_dir}")

# You can then read it back
# reimported_agent = AgentUtils.create_agent_from_directory_structure(export_dir, "ProgrammaticAgent")
# print(f"Re-imported agent name: {reimported_agent.name}")